In [ ]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
train = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/sample_submission.csv')
test.head()

In [ ]:
print("Кол-во классов: {}".format(train['emotion'].nunique()))

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = 48

image_gen = ImageDataGenerator() 

train_generator = image_gen.flow_from_dataframe(train,
                                                 directory=str('/kaggle/input/train-data'),
                                                 x_col='image_path',
                                                 y_col='emotion',
                                                 batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                class_mode='categorical')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization 

model = Sequential([
# 1 сверточный слой
    Conv2D(64, (5, 5), input_shape=(48,48,3), activation='relu', padding='same'),
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    BatchNormalization(), # Слой нормализации данных
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5), # Необходим для предотвращения переобучения

# 2 сверточный слой
    Conv2D(128, (5, 5),activation='relu',padding='same'),
    Conv2D(128, (5, 5),activation='relu',padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

# 3 сверточный слой
    Conv2D(256, (3, 3),activation='relu',padding='same'),
    Conv2D(256, (3, 3),activation='relu',padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Flatten(), # Конвертация в двумерный тензор
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(9, activation='softmax')
])
my_optimiser = tf.keras.optimizers.Adam(
                    learning_rate=0.001, 
                    beta_1=0.9, 
                    beta_2=0.999, 
                    epsilon=1e-07, 
                    amsgrad=False,
                    name='Adam')

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=my_optimiser)


In [ ]:
%%timeit -n 10 -r 10

test = model(np.random.rand(1,48,48,3).astype(np.float32))

In [ ]:
history = model.fit(train_generator, epochs=10, batch_size=BATCH_SIZE)

In [ ]:
predict_dict = {0:'anger',1:'contempt',2:'disgust',3:'fear',4:'happy',5:'neutral',6:'sad',7:'surprise',8:'uncertain'}

In [ ]:
test_data = tf.keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/test-data',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE,IMAGE_SIZE)
)

In [ ]:
emotion = model.predict(test_data)

In [ ]:
emotion_cm = [predict_dict[np.argmax(i)] for i in emotion]

In [ ]:
test['emotion'] = emotion_cm
test.head()

In [ ]:
test.to_csv('/kaggle/working/submission.csv', sep=',', index=False)

В этом ноутбуке я использовал 3-ую свертку. Здесь осталась ошибка с неправильным датасетом тестовых данных. Score на обучающих данных: 0.4434